# Preprocesamiento

Se importan las librerías a utilizar

In [ ]:
import pandas as pd
import tensorflow as tf
import numpy as np
from tensorflow import keras
from keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn import metrics
from sklearn.metrics import r2_score

Se configura la GPU

In [ ]:
# Lista de GPUs físicas disponibles
gpus = tf.config.experimental.list_physical_devices('GPU')

if gpus:
    try:
        # Configurar una GPU virtual con 6.8 GB de memoria
        tf.config.experimental.set_virtual_device_configuration(
            gpus[0],
            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=6800)])  # 6.8 GB en megabytes

        # Verificar si la configuración se realizó correctamente
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "GPU física(s),", len(logical_gpus), "GPU(s) lógica(s)")

    except RuntimeError as e:
        print(e)

Se crea el dataframe

In [ ]:
# Carga tus datos desde un archivo CSV (ajusta la ruta según la ubicación de tus datos)
data = pd.read_csv('..\Semana 17\datos_clusterizados.csv', dtype={'rut': str})
data

Se borran las columnas que no serán utilizadas

In [ ]:
data.drop('rut',axis=1,inplace=True)
data.drop('ano',axis=1,inplace=True)
data

Se codifican las variables categóricas

In [ ]:
le_rangoEdad = LabelEncoder()
le_plan = LabelEncoder()
le_fidelizacion = LabelEncoder()

In [ ]:
rangoEdad_encoded = le_rangoEdad.fit_transform(data['rangoEdad'])
plan_encoded = le_plan.fit_transform(data['plan'])
fidelizacion_encoded = le_fidelizacion.fit_transform(data['fidelizacion'])
cluster = data['cluster']

encoded_df = data.drop(['rangoEdad','plan','fidelizacion','cluster'], axis=1)
encoded_df.insert(0, column='rangoEdad_encoded', value=rangoEdad_encoded)
encoded_df.insert(1, column='plan_encoded', value=plan_encoded)
encoded_df.insert(2, column='fidelizacion_encoded', value=fidelizacion_encoded)
encoded_df.insert(3, column='cluster', value=cluster)

encoded_df

Se definen las variables a utilizar para entrenar (x) y las variables esperadas como predicción (y)

In [ ]:
features = encoded_df.drop(['visitasHospital_2022', 'visitasAmbulatorio_2022', 'visitasDiagnostico_2022', 'visitasDental_2022'], axis=1)
labels = encoded_df[['visitasHospital_2022', 'visitasAmbulatorio_2022', 'visitasDiagnostico_2022', 'visitasDental_2022']]

Se dividen los datos en sets de entrenamiento y prueba

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

Se escala la data

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Red neuronal

Se crea el modelo de la red neuronal

In [ ]:
# Define el modelo de red neuronal
model = keras.Sequential([
    keras.layers.Dense(128, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(4, activation='softmax')  # 4 salidas para predecir 'visitasHospital_2022', 'visitasAmbulatorio_2022', 'visitasDiagnostico_2022', 'visitasDental_2022' 
])

Se muestra un resumen del modelo creado anteriormente

In [ ]:
model.summary()

Se compila la red neuronal

In [ ]:
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy']) 

Se entrena la red neuronal

In [ ]:
model.fit(X_train_scaled, y_train, epochs=50, batch_size=10000, validation_data=(X_test_scaled, y_test))

Evaluación del modelo en los datos de prueba del 2022

In [ ]:
loss = model.evaluate(X_test_scaled, y_test)
print(f'Pérdida en datos de prueba del 2022: {loss}')

Predicciones en los datos de prueba (2022)

In [ ]:
predictions = model.predict(X_test_scaled)
predictions

In [ ]:
MSE = np.sqrt(metrics.mean_squared_error(y_test,predictions))
R2 = r2_score(y_test,predictions)
print('MSE: ',MSE)
print('R2_Score: ',R2)

# Post procesamiento

Crea un nuevo objeto StandardScaler solo para las 4 columnas de predicciones, para poder invertir el escalamiento de la predicción

In [ ]:
scaler_for_predictions = StandardScaler()
scaler_for_predictions.fit(data[['visitasHospital_2022', 'visitasAmbulatorio_2022', 'visitasDiagnostico_2022', 'visitasDental_2022']])

Se revierte el escalado de las predicciones

In [ ]:
predictions_original_scale = scaler_for_predictions.inverse_transform(predictions)
predictions_original_scale

Se crea un dataframe para poder ver la predicción de mejor manera

In [ ]:
pred_df = pd.DataFrame(predictions_original_scale,columns=['%_visHosp2022', '%_visAmb2022', '%_visDiag2022', '%_visDent2022'])
pred_df

In [ ]:
final_data = X_test
final_data.reset_index(drop=True,inplace=True)

inverse_rangoEdad = le_rangoEdad.inverse_transform(final_data['rangoEdad_encoded'])
final_data.insert(0, column='rangoEdad', value=inverse_rangoEdad)
final_data.drop('rangoEdad_encoded',axis=1,inplace=True)

inverse_plan = le_plan.inverse_transform(final_data['plan_encoded'])
final_data.insert(1, column='plan', value=inverse_plan)
final_data.drop('plan_encoded',axis=1,inplace=True)

inverse_fidelizacion = le_fidelizacion.inverse_transform(final_data['fidelizacion_encoded'])
final_data.insert(2, column='fidelizacion', value=inverse_fidelizacion)
final_data.drop('fidelizacion_encoded',axis=1,inplace=True)

final_df = pd.concat([final_data, pred_df],axis=1)
final_df.dropna(inplace=True)
final_df.reset_index()
final_df

In [ ]:
final_df

In [ ]:
print("Probabilidad máxima de visitas de hospital en el 2022: ", final_df['%_visHosp2022'].max())
print("Probabilidad máxima de visitas de ambulatorio en el 2022: ", final_df['%_visAmb2022'].max())
print("Probabilidad máxima de visitas de diagnóstico en el 2022: ", final_df['%_visDiag2022'].max())
print("Probabilidad máxima de visitas de dental en el 2022: ", final_df['%_visDent2022'].max())

In [ ]:
hosp = final_df[final_df['%_visHosp2022'] == final_df['%_visHosp2022'].max()]
hosp

In [ ]:
amb = final_df[final_df['%_visAmb2022'] == final_df['%_visAmb2022'].max()]
amb

In [ ]:
diag = final_df[final_df['%_visDiag2022'] == final_df['%_visDiag2022'].max()]
diag

In [ ]:
dent = final_df[final_df['%_visDent2022'] == final_df['%_visDent2022'].max()]
dent